In [58]:
import xml.etree.ElementTree as ET
import pandas as pd
tree = ET.parse('NEWS2018_M-EnHi_trn.xml')
 
    
# getting the parent tag of
# the xml document
root = tree.getroot()
 

FileNotFoundError: [Errno 2] No such file or directory: 'NEWS2018_M-EnHi_trn.xml'

In [13]:
len(root)

12937

In [ ]:
tree = ET.parse('NEWS2018_M-EnHi_trn.xml')
 
# getting the parent tag of
# the xml document
root = tree.getroot()

english = []
hindi = []


for i in range(len(root)):
    english.append(root[i][0].text)
    hindi.append(root[i][1].text)

In [ ]:
df = pd.DataFrame({"english":english,"hindi":hindi})

In [ ]:
df.to_excel('train_data.xlsx',index=False)

In [52]:
tree = ET.parse('NEWS2018_M-EnHi_dev.xml')
 
# getting the parent tag of
# the xml document
root = tree.getroot()

english = []
hindi = []


for i in range(len(root)):
    english.append(root[i][0].text)
    hindi.append(root[i][1].text)

In [ ]:
df = pd.DataFrame({"english":english,"hindi":hindi})

In [ ]:
df.to_excel('test_data.xlsx',index=False)

In [1]:
from torchtext.data.metrics import bleu_score

In [ ]:
def bleu(data, model, english, hindi, device):
    targets = []
    outputs = []

    for (src,trg) in tqdm(data):
        for src_text, trg_text in zip(src.permute(1,0),trg.permute(1,0)):
            trg_text = [hindi.itos[idx.item()] if idx.item() in hindi.itos else hindi.itos[3] for idx in trg_text]
            trg_text = [i for i in trg_text if i != '<PAD>']

            prediction = translate_sentence(model, src_text, english, hindi, device)
            prediction = prediction[:-1]  # remove <eos> token

            targets.append([trg_text])
            outputs.append(prediction)

    return bleu_score(outputs, targets)

In [5]:
bleu_score(list("दावूद"),list("दावूवूवूद"))

AssertionError: The length of candidate and reference corpus should be the same

In [31]:
import nltk

hypothesis = list("दावूद")
reference = list("दावूद")
#the maximum is bigram, so assign the weight into 2 half.
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, weights = (0.5, 0.5))
print(round(BLEUscore,3))
print(BLEUscore)

1.0
1.0


In [17]:
import pandas as pd

In [72]:
df = pd.read_excel('input/validate_df.xlsx')

In [73]:
df.head()

,english,hindi
0,aachaaryanandana,आचार्यनंदना
1,aachaarysut,आचार्यसुत
2,aacharynandan,आचार्यनंदन
3,aacharynandanaa,आचार्यनंदना
4,aadamkhor,आदमखोर


In [74]:
from tqdm import tqdm
hindi_predicted = []
bleu_score = []
for i in tqdm(range(len(df))):
    eng = df.iloc[i,0]
    hin = df.iloc[i,1]
    hin_pre = predict(eng)
    hindi_predicted.append(hin_pre)
#     BLEUscore = nltk.translate.bleu_score.sentence_bleu([list(hin_pre)], list(hin), weights = (0.5, 0.5))
#     bleu_score.append(round(BLEUscore,3))

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:18<00:00, 52.82it/s]


In [33]:
sum(bleu_score)/len(bleu_score)

0.6942519999999986

In [38]:
from sklearn.metrics import f1_score

In [75]:
df['hindi_predicted'] = hindi_predicted

In [76]:
df.head()

,english,hindi,hindi_predicted
0,aachaaryanandana,आचार्यनंदना,आचार्यनंदना
1,aachaarysut,आचार्यसुत,आचार्यसुत
2,aacharynandan,आचार्यनंदन,आचार्यनंदन
3,aacharynandanaa,आचार्यनंदना,आचार्यनंदना
4,aadamkhor,आदमखोर,आडाघोर


In [77]:
f1_sc = []
for i in range(len(df)):
    tru_hin = df.iloc[i,1]
    pre_hin = df.iloc[i,2]
    true_hind = list(tru_hin.ljust(max(len(tru_hin),len(pre_hin)), ' '))
    pred_hind = list(pre_hin.ljust(max(len(tru_hin),len(pre_hin)), ' '))
#     print(true_hind)
#     print(pred_hind)
    f1_sc.append(f1_score(true_hind,pred_hind,average='macro'))

In [78]:
df['f_score'] = f1_sc

In [79]:

mean(f1_sc)

0.5765055586569781

In [71]:
df.to_excel('output/train_prediction.xlsx',index=False)

In [55]:
f1_sc = []
for i in range(len(df)):
    true_hind = list(df.iloc[i,1].ljust(100, ' '))
    pred_hind = list(df.iloc[i,2].ljust(100, ' '))
#     print(true_hind)
#     print(pred_hind)
    f1_sc.append(f1_score(true_hind,pred_hind,average='micro'))

In [60]:

mean(f1_sc)

0.6372632265852266

In [54]:
from statistics import mean

mean(f1_sc)

0.6372632265852266

In [44]:
f1_score(list(df.iloc[4,1]),list(df.iloc[4,2]),average='macro')

0.3333333333333333

In [25]:
import requests

def predict(msg):
    url = "http://127.0.0.1:5000/predict"

#     payload="{\r\n   \"message\":\"tata\"\r\n}"
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, json={"message":msg})

    return response.json()['transliterated_text']


In [26]:
predict("bye")

'बेये'